<a href="https://colab.research.google.com/github/DayoungKwon/mrc/blob/main/mrc_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 44.8 MB/s 
     |████████████████████████████████| 212 kB 56.9 MB/s 
     |████████████████████████████████| 163 kB 52.7 MB/s 
     |████████████████████████████████| 127 kB 58.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### Dataset 살펴보기 
- train, valid, test 구성


In [3]:
from pprint import pprint

print('train dataset 살펴보기 ----')
print(dataset['train'])
pprint(dataset['train'][0])

train dataset 살펴보기 ----
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 60407
})
{'answers': {'answer_start': [54], 'text': ['교향곡']},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 '
            '갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 '
            '메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 '
            '연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 '
            '작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 '
            '것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. '
            '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 '
            '동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 '
            '않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 '
            '말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, '
            '그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가

In [4]:
answer_start = dataset['train'][0]['answers']['answer_start'][0]
print(dataset['train'][0]['context'][answer_start:answer_start+10])

교향곡을 쓰려는 뜻


### Baseline 살펴보기 
Konlpy: 파이썬 한국어 NLP 처리기  
Mecab-korean: 한국어 형태소 분석기  

https://www.kaggle.com/code/mynsng/korean-mrc-baseline-goorm-5

In [5]:
! apt-get install -y openjdk-8-jdk python3-dev
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0

In [16]:
train_sample = {
  "version": "Ko-MRC",
  "data": [
    {
      "title": "제주도 장마 시작 … 중부는 이달 말부터",
      "paragraphs": [
        {
          "context": "올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.",
          "qas": [
            {
              "question": "북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?",
              "answers": [
                {
                  "text": "한 달가량",
                  "answer_start": 478
                },
                {
                  "text": "한 달",
                  "answer_start": 478
                }
              ],
              "guid": "798db07f0b9046759deed9d4a35ce31e"
            }
          ]
        }
      ],
      "news_category": "종합",
      "source": "hankyung"
    },
    {
      "title": "부산정보산업진흥원, 과기부 지역SW서비스사업화 지원사업 4개 과제 선정",
      "paragraphs": [
        {
          "context": "부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리체계 개발을 목표로 한다. ○ 엔컴(주)이 주관기업으로 참여하는 <AI영상분석 기반 가공철근 생산성 향상 시스템 기술개발 및 사업화>는 산업안전, 환경규제, 생산체계의 변화로 침체된 부산 핵심 산업인 철강업 활성화에 나선다. 실시간으로 절곡되어 나오는 가공철근의 형상을 인식하고 불량 형상 판단 시 적합한 교정 값을 절곡설비에 전달함으로써, 무중단 생산이 가능한 영상분석 기술과 생산설비 자동화 제어기술을 개발한다. ○ ‘Track 2’의 ㈜토즈는 자립기반이 약한 국내 중소형 조선소의 산업기술 변화에 혁신적인 대응을 위해 <가상현실 기반 원격 다자간 선박 및 해양구조물 사전 검사 시스템>을 개발한다. 선박 건조 前, 설계 단계에서 설계자 뿐만 아니라 생산관리자, 품질관리자, 선급검사관, 선주감독관 등의 이해관계자가 공동으로 가상의 환경에서 선박 및 해양구조물의 자재 배치와 간섭, 작업성, 설계 오작 등에 대한 검사를 진행할 수 있는 기술을 확보하여 조선소의 업무효율을 극대화 할 예정이다. ○ 삼보테크놀로지는 재래식 건설 부자재의 시공성, 안전성, 내구성 등의 문제점을 보완하여 시민 안전과 건설근로자의 환경개선, 생산성 및 수익성 향상을 위해 <인공지능융합센서와 새들형 토치 서보 이송 로봇을 이용한 고속 SRD 전단보강재 자동용접시스템>을 개발한다. 로봇응용 SRD 용접자동화 설비를 제작하고, 용접 모니터링 및 품질검사 소프트웨어를 개발하여 건설분야에 4차산업 대비 지능형 생산자동화 기반기술을 확보할 예정이다. (재)부산정보산업진흥원 이인숙 원장은 “이번 코로나19 사태로 인해 부산 기업들이 매출과 고용유지, 자재수급 등에 큰 타격을 입었지만, 지역SW서비스사업화 지원사업을 통해 지역과 기업차원에서 기반을 다지는 계기가 됐으면 좋겠다‘며 ”진흥원은 어려운 사태를 대비해 지역 기업들을 지원할 수 있는 다른 방편을 계속 모색 중이며, 더욱 성장해 나갈 수 있도록 적극 지원하겠다“고 전했다.",
          "qas": [
            {
              "question": "지능형 생산자동화 기반기술을 개발중인 스타트업은?",
              "answers": [
                {
                  "text": "삼보테크놀로지",
                  "answer_start": 1422
                }
              ],
              "guid": "67c85e4f86ae43939b807684537c909c"
            }
          ]
        }
      ],
      "news_category": "경제",
      "source": "acrofan"
    },
    {
      "title": "나루세 요시히사",
      "paragraphs": [
        {
          "context": "시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으로서는 처음으로 개막전 선발 투수로 지명되었다. 요코하마 고등학교의 후배이기도 한 사이타마 세이부 라이온스의 와쿠이 히데아키도 개막전 선발로 등판해 5이닝까지 노히트로 처리하였지만 7회에 2점 홈런을 허용하면서 역전당했다. 타선에서도 1점을 따는데만 그쳐 3안타 2실점을 기록하여 패전 투수가 되었다. 그 외에도 다르빗슈 유, 이와쿠마 히사시도 개막전 선발 투수로서 등판하였지만 모두 패전 투수가 되었기 때문에 개막전 투수가 3명 모두 완투패를 당한 것이어서 50년 만에 일본 프로 야구에서의 진기록이 나왔다.\n\n정규 시즌에서는 팀내 최다인 13승을 올려 203.2 이닝의 완투 7개, 무볼넷 경기 4차례, 탈삼진 192개, WHIP은 1.02의 좋은 성적을 남겼다. 그러나 피홈런 개수가 증가되면서 29개가 나와 11패를 기록했다. 오릭스 버펄로스 전에서는 7전 7승이라는 기대 이상의 성적을 나타냈고 최종전에서도 승리해 팀을 클라이맥스 시리즈 진출에 기여했다.\n\n클라이맥스 시리즈 파이널 스테이지에서는 후쿠오카 소프트뱅크 호크스와 상대하면서 시즌에서는 평균자책점이 3.32였지만 0승 4패라는 최악의 성적을 남겼다. 그러나 퍼스트 스테이지 1차전(세이부전, 7과 0/3이닝, 2실점)에서 1실점 완투 승리를 거뒀고 최종전인 6차전에서도 4개의 피안타와 완봉 승리를 거두는 등 클라이맥스 시리즈 사상 시즌 3위였던 팀이 처음으로 일본 시리즈 진출에 기여하여 자신도 파이널 스테이지 MVP에 선정되었다. 일본 시리즈에서도 1차전과 6차전에 선발 등판해 1차전에서는 2실점 호투하여 승리해 자신으로서는 처음으로 일본 시리즈의 승리 투수가 되었다.",
          "qas": [
            {
              "question": "개막전에서 3안타 2실점을 기록해서 패한 선수는?",
              "answers": [
                {
                  "text": "와쿠이 히데아키",
                  "answer_start": 107
                }
              ],
              "guid": "d2764543b0a84596942b34071541bed4"
            }
          ]
        }
      ],
      "news_category": "",
      "source": "wikipedia"
    },{
      "title": "에티하드항공, ‘최고의 이코노미 클래스 및 상용 고객 프로그램 상‘ 수상",
      "paragraphs": [
        {
          "context": "아랍 에미리트의 국영항공사 에티하드항공이 지난 10월 21일 열린 ’2020비즈니스 트래블러 중동 어워드(Business Traveller Middle East Awards 2020)에서 ‘최고의 이코노미 클래스(Best Economy Class)’와 ‘최고의 상용고객 프로그램(Best Frequent Flyer Programme’)’상의 영예를 안았다. 이는 2012년과 2013년에 열린 비즈니스 트래블러 중동 어워드에서 각각 최고의 이코노미 클래스, 최고의 상용 고객프로그램 분야의 첫 수상을 한 이래로 2회째 수상 기록이다. 지난 해, 에티하드항공은 다양한 수상 경력을 지닌 비즈니스, 퍼스트, 더 레지던스 캐빈에서 펼쳐 온 혁신과 마찬가지로 이코노미석에서도 기내식 선택권을 넓히고 무선 기내 엔터테인먼트, 인체공학적 디자인으로 설계된 좌석과 여유로운 레그룸을 가진 이코노미석 도입을 비롯해 옆 좌석 비우기, 보다 넉넉한 이코노미 스페이스 석 선택 옵션, 선호좌석 선택 사항을 제공하는 등 기내 경험에 대한 더욱 다양한 선택권을 제공해 왔다. 이를 통해 승객이 개인의 필요와 요구에 맞는 맞춤화된 선택을 할 수 있도록 돕고 만족도 높은 항공 여행을 할 수 있도록 서비스와 상품 개선에 투자를 지속해 왔다. 또한, 에티하드항공은 항공사의 건강 및 위생 프로그램인 ‘에티하드 웰니스’프로그램을 통해 신종코로나바이러스로 인한 전례 없는 시기에 필수적인 여행을 해야만 하는 여행자들에게 마음의 평화와 여행에 대한 자신감을 심어주기 위해 펼친 다양한 노력이 고객의 긍정적인 평가를 받았다고 전했다. 예약 과정에서부터 공항 이용은 물론 항공여행에 이르기까지 인공지능 기술을 비롯한 최신 기술을 과감히 도입해 광범위한 예방 조치를 시행하고 있으며 업계 최초로 선보인 에티하드 웰니스 프로그램에서는 특별 훈련 과정을 거친 에티하드항공의 웰니스 엠버서더가 여행 전 과정에 걸친 조언과 건강 및 위생 조치에 대한 세부 사항을 공유하며 맞춤화된 정보를 제공한다. 한 해 동안, 에티하드항공은 에티하드 게스트 회원의 경험을 향상시키고 혜택의 범위를 확장함으로써 더 나은 가치를 선사하기 위한 노력을 펼쳐왔다. 최근에는 “당신의 특별한 손님(Your Special Guest)” 캠페인과 함께 삶의 소중한 사람을 지정해 선물을 보내주는 캠페인을 펼치며 비대면의 시대에서도 따뜻한 마음과 정을 표현할 수 있는 기회를 제공했다. 더불어 코로나19로 인한 팬데믹 상황을 고려해 추가 마일리지를 지급하거나 항공권 예약 및 변경에 있어 유연한 정책 도입을 도입함은 물론 파트너십의 범위를 확장해 회원 등급을 유지하기 위한 마일리지를 적립할 수 있는 방법을 다양화 했다. 한편, 신종코로나바이러스 상황으로 인해 올해 2020비즈니스 트래블러 중동 어워드는 온라인으로 개최되었으며 독자의 투표를 기반으로 선정됐다.",
          "qas": [
            {
              "question": "에티하드 웰니스 프로그램의 일환으로 위생에 관한 정보를 제공하는 것은 누구인가?",
              "answers": [
                {
                  "text": "웰니스 엠버서더",
                  "answer_start": 913
                }
              ],
              "guid": "51c829337dbb4a748c7ad2deda505aec"
            },
            {
              "question": "에티하드항공이 최고의 이코노미 클래스 상을 두 번째로 받은 해는?",
              "answers": [
                {
                  "text": "2020",
                  "answer_start": 38
                }
              ],
              "guid": "24c31e1ccee54b90b279e9866317f17f"
            }
          ]
        }
      ],
      "news_category": "산업",
      "source": "acrofan"
    },
    {
      "title": "특집 인터넷 LIVE AI 인재 100만 강국, 더 큰 대한민국! 제3차 국회 ICT 융합포럼 개최",
      "paragraphs": [
        {
          "context": "방송일시(인터넷 LIVE) : 2020년 12월 10일 (목) 낮 15:00 – 17:10 방송채널 : 유튜브(EBSlearning), 이솦(EBS 소프트웨어 사이트) 4차 산업혁명의 핵심 성장 동력인 AI는 이미 사회 전반에 걸쳐 광범위한 영향을 미치고 세계 산업과 사회를 혁신시키는 중요한 역할을 하고 있다. 그러나 전 세계 인공지능 핵심 인재 500명 가운데 우리나라 출신 비율은 1.4%에 불과한 실정이다. 이는 미국(14.6%)과 중국(13.0%)의 10분의 1 수준으로, 싱가포르(6.2%), 프랑스(2.8%), 대만(1.8%), 이스라엘(1.6%)보다도 낮다. 10일 오후 3시 EBS 스페이스 홀에서 열리는 제3차 국회 ICT 융합포럼에서는 AI 인재 100만 강국, 더 큰 대한민국! 을 주제로 미래 핵심기술로서의 AI 산업 경쟁력을 높이고 인재 양성을 지원하는 방안을 모색한다. 한편 이번 포럼은 국회 ICT 융합포럼(대표의원 : 변재일 더불어민주당 의원•조명희 국민의힘 의원)과 국회 국토공간정보정책포럼, EBS(사장 김명중)가 공동으로 주최하며 코로나19 확산 방지를 위해 EBS 유튜브(EBSLearning) 및 EBS 이솦을 통해 전국의 AI 전공 대학 및 대학원생을 대상으로 생중계 될 예정이다. 국회 ICT 융합포럼 대표의원인 조명희 의원(국민의힘)의 개회사를 시작으로, 대표의원인 변재일 의원(더불어민주당)의 환영사, 그리고 박병석 국회의장, 여야 당 대표 및 원내대표의 축사에 이어 기조연설과 주제발표가 진행된다. 기조연설은 인공지능 분야에서 가장 권위 있는 학회인 AAAI 학술대회(AAAI-2015)에서 ‘상상력 기계(Imagination Machine)’를 발표하여 세계적 주목을 받았던 장병탁 서울대 AI 연구원장이 <대한민국에는 어떤 AI 인재가 필요한가?>를 주제로 한국판 뉴딜의 성공을 위한 AI․인재 양성의 중요성과 육성방안을 제시한다. 주제발표는 <AI기술과 교육>, <AI산업과 전망>을 중심으로 함진호 한국전자통신연구원 전문연구위원(AI 아카데미 준비 TF 위원장), 김준석, 현대자동차 AIR LAB 실장, 김채희 KT AI • 빅데이터사업본부장, 하정우 네이버 AI LAB 연구소장이 연사로 나서서 현재 우리나라AI 교육현황과 기업의 AI 기술 전략을 발표한다. 한편 국회 ICT 융합포럼 대표의원인 조명희 의원(국민의힘)이 좌장으로 진행하는 주제토론에는 홍성완 과장(과학기술정보통신부 소프트웨어정책과) 등 정책 담당자와 이기준교수(부산대 컴퓨터공학부), 성미영교수(인천대 켬퓨터공학부) 등 학계전문가, 김광범 EBS 학교교육본부장, 김민준 대표(AI클라우드 벤처기업)가 참여해 AI 인재양성을 위한 교육정책과 협력방안을 모색한다. 특히 성미영교수(인천대 컴퓨터공학부)는 우수한 여성 IT 인재들이 재능을 낭비하고 사라져 버리는 현실에 대한 개선방안을, 김광범 EBS 학교교육본부장은 인공지능 교육 활성화를 위한 소프트웨어 교육과 ICT 활용 교육의 중요성을 그리고 홍성완 과장(과학기술정보통신부 소프트웨어정책과)은 한국판 뉴딜인 AI․ SW 핵심 인재 10만 양성을 위한 핵심 사업과 구체적인 추진 과제를 제시한다. 행사를 주관한 국회 ICT 융합포럼 대표의원인 조명희 의원(국민의힘)은 이번 포럼에 발표된 내용을 토대로 미래 핵심기술인 AI 산업 경쟁력을 높이고 인재 양성 지원 정책을 발굴하여 , 코로나 이후 본격화 될 ‘비욘드 사피엔스’ 시대를 준비하는 데 앞장서겠다고 각오를 밝혔다.",
          "qas": [
            {
              "question": "2015년에 열렸던 AAAI에 논문을 개재한 사람의 소속 학교는?",
              "answers": [
                {
                  "text": "서울대",
                  "answer_start": 858
                }
              ],
              "guid": "6079f21ec9604cd6ac0ce356eaaea828"
            }
          ]
        }
      ],
      "news_category": "미디어",
      "source": "acrofan"
    },
    {
      "title": "코로나 19 심각 단계 격상에 따라 2020 정동극장 레퍼토리 적벽, 3월 8일까지 잠정 중단",
      "paragraphs": [
        {
          "context": "(재)정동극장(대표이사 김희철)은 정부가 23일 코로나 19 위기 경보를 ‘심각’ 단계로 높임에 따라 이에 대한 조치 사항으로 2월 14일 개막한 공연 <적벽>을 3월 8일까지 잠정 중단하기로 결정했다. 정동극장 레퍼토리 <적벽>은 오는 4월 5일까지 공연이 예정되어 있었다. 이번 <적벽> 공연 잠정 중단 결정에 따라 남은 기간 공연 예매자 환불 조치 등 후속 조치가 진행 된다. 정동극장은 ‘코로나 19’ 예방 조치를 위해 그동안 정기 방역 작업을 주 1회로 확대 실시하고, 공연장 로비에 열 감지기를 도입하였으며, 공연장 곳곳에 손 소독제 설치와 비 접촉식 체온계 및 관객 배포용 마스크를 비치하는 등 최선의 예방조치를 다 해 왔다. 그러나 정부 대응 단계가 ‘심각’으로 격상됨에 따라 사태의 심각성을 고려해 조치를 쥐하게 되었다. 한편, <적벽>은 2월 14일 첫 공연 이후, 좋은 관객 반응과 티켓 판매율 성장세를 보이고 있었다. 정동극장은 상한가 분위기 속 <적벽> 공연을 3월 8일까지 중단한 결정에 대해 “현재 감염병 위기경보 최고 수준인 ‘심각’ 단계로 정동극장은 추가 확진자 발생 및 2차 감염에 대한 우려로 국민의 건강과 안전을 최우선으로 고려해 깊은 고심 끝에 결정을 내렸다”고 그 이유를 설명했다. 3월 8일 이후 공연 <적벽> 재개에 대한 부분은 향후 상황 추이를 지켜보며, 정부 대응을 살펴 결정할 방침이다. 또한, 정동극장은 앞으로도 ‘코로나 19’ 예방 및 확산 방지를 위해 할 수 있는 최선을 다 할 것이라고 밝혔다.",
          "qas": [
            {
              "question": "코로나19 심각 단계 선언 전에 계획했던 적벽의 마지막 공연일은?",
              "answers": [
                {
                  "text": "4월 5일",
                  "answer_start": 133
                }
              ],
              "guid": "3c496c72ba574d6ea55bdc7a85ff02f2"
            }
          ]
        }
      ],
      "news_category": "생활",
      "source": "acrofan"
    },
    {
      "title": "삼성의 저력, 스파르타식 임원 교육",
      "paragraphs": [
        {
          "context": "“제트기가 음속을 돌파하려면 비행기의 모든 소재가 다 바뀌어야 한다. 재료공학부터 기초물리, 화학까지 첨단기술이 총동원돼야 한다. 설계도는 물론이고 엔진과 부품을 완전히 바꿔야 한다.”6일 경기도 용인 삼성인력개발원에 모인 200여 삼성그룹 임원들은 이건희 회장이 7년 전 강조한 ‘마하경영’의 의미를 되새겼다. 김동재 연세대 국제학대학원 교수의 마하경영 강연에 이어 분임조별 토론이 벌어졌다. 이틀 동안 진행된 교육에서 강연과 토론에 할애된 시간만 20시간이 넘었다. 삼성이 매년 2월 진행하는 임원진 교육 현장이다. 삼성은 그룹 내 2000여명의 임원을 10개 조로 나눠 2월 한 달 동안 1박2일간 합숙 교육한다.올해는 한계돌파를 위한 마하경영에 임원 교육의 초점이 맞춰졌다. 지난 연말 열린 사장단 세미나와 지난달 신임 임원 교육에 이어 2월 기존 임원진 교육에서도 공통된 주제는 마하경영이다. 하나의 방향을 정하면 그대로 실행에 옮기는 삼성의 스파르타식 교육의 한 단면이다.이날 강연을 들은 한 임원은 “그룹 차원에서 설정한 방향과 가치를 공유하며 정신을 무장하는 자리”라며 “현업에서 이를 어떻게 실천할 것인지 고민해보는 시간을 가졌다”고 말했다. 그는 “삼성의 힘은 결국 이런 교육에서 나온다”고 덧붙였다. 이날 마하경영을 설명한 김 교수는 지난달 신임 임원 교육에서도 강연자로 나섰다. ‘마하(Mach)’는 음속 제트기의 속도를 측정하는 단위다. 그는 “음속을 돌파하려면 부분 개보수가 아니라 자기부정부터 시작해 행동부터 생각까지 근본적으로 바뀌어야 한다”고 강조했다.이 회장은 2006년 당시 전자 계열사 사장들에게 “삼성의 약점을 보완하고 성장하려면 마하1이나 2가 아니라 마하3은 돼야 한다”며 “현재 삼성은 음속 이하의 수준인 만큼 진정한 글로벌 선진 기업이 되려면 더 분발해야 한다”고 했다.1993년 신경영 선언 당시 “보잉747은 이륙할 때 몇분 만에 1만ｍ까지 올라가야 하는데 중간에 멈추면 그대로 추락하거나 공중폭발을 하고 만다”는 ‘비행기 이륙’에서 ‘제트기 음속’으로 진화한 개념이다. 올해 다시 등장한 마하경영은 속도에서 가치 향상으로의 지속적인 혁신에 중점을 뒀다. 1등을 빨리 따라잡아야 한다는 추격자에서 이젠 세계 1등 기업에 맞는 체질로의 변신을 요구하고 있는 것이다. 마하경영과 함께 2등을 확실히 따돌릴 정도의 차이를 벌릴 때까지 안심해서는 안 된다는 ‘초격차’와 변화의 주도권을 잡기 위한 ‘한계 돌파’도 화두로 제시됐다. 올해 승진한 300여 신임 임원들은 지난달 5박6일간 마하경영 교육과 함께 임원으로서의 기본소양과 리더십 교육을 받았다. 다른 기업들과 달리 삼성은 직원들 교육만큼 임원 교육 비중이 크다.신임임원 교육으로 입문한 이후엔 매년 1박2일간의 임원 교육에 참석해야 하고 부사장급이 되면 고위경영자 양성교육 후보군에 들어간다. 이는 미래 최고경영자(CEO) 후보들을 위한 핵심인재 교육으로 연 2회로 4주에 걸쳐 진행된다.이후 사장이 되면 매주 수요일 사장단회의에서 강연을 듣고 매년 말 다음해의 전략을 세우고 점검하는 1박2일간의 사장단 세미나에 참석해야 한다. 삼성그룹 관계자는 “인재 양성뿐 아니라 그룹의 핵심가치를 공유하고 전파하는 것도 교육”이라며 “삼성만의 DNA를 심기 위한 인재 관리의 핵심”이라고 말했다.",
          "qas": [
            {
              "question": "지난달 신임 임원 교육에서 강연했던 사람의 이름은?",
              "answers": [
                {
                  "text": "김동재",
                  "answer_start": 175
                }
              ],
              "guid": "f7c95fd5471a4e3eb8f25ffb2741f60f"
            }
          ]
        }
      ],
      "news_category": "산업섹션",
      "source": "hankyung"
    },
    {
      "title": "넥슨, ‘2020 카트라이더 리그 시즌2’ 일정 공개",
      "paragraphs": [
        {
          "context": "㈜넥슨(대표 이정헌)은 온라인 레이싱게임 ‘카트라이더’에서 진행하는 e스포츠 대회 ‘2020 카트라이더 리그 시즌2’ 일정을 공개했다고 31일 밝혔다. 29번째 정규 대회 ‘2020 카트라이더 리그 시즌2’는 8월 22일 첫 본선 경기로 막을 올린다. 이후 11월 7일 결승전까지 12주 동안 매주 수요일, 토요일 오후 6시에 무관중으로 진행된다. 넥슨은 이번 시즌부터 CJ ENM 게임채널 OGN과 손잡는다. 서울 상암동 OGN e스타디움에서 무관중 경기가 운영되며, 주관 방송사인 OGN TV채널에서 모든 경기를 시청할 수 있다. 또 본선 팀전 진행방식을 기존 8강, 4강 전체 풀리그에서 8강 풀리그 이후 포스트시즌으로 변경한다. 포스트시즌 방식을 통해 와일드카드전, 준플레이오프 등 매 경기마다 더욱 치열한 승부가 펼쳐질 수 있도록 마련한다. 넥슨은 ‘2020 카트라이더 리그 시즌2’ 개막에 앞서 8월 1, 2일 양일 온라인 예선을 진행한다. 이날 예선전에서는 그랑프리 점수에 따라 선발된 32개 팀과 128명의 개인 선수가 참가해 본선 진출권을 두고 경합한다. 이 중 팀 지원 프로젝트 모집 결과 및 지난 시즌 최종 순위에 따라 ‘한화생명e스포츠’, ‘샌드박스 게이밍’, ‘성남 락스’, ‘아프리카 프릭스’ 팀과 박인수 선수는 시드를 제공받아 본선에 직행한다. ‘2020 카트라이더 리그 시즌2’에 대한 자세한 내용은 공식 홈페이지에서 확인할 수 있다.",
          "qas": [
            {
              "question": "2020 카트라이더 리그 시즌2'의 결승전은 언제인가?",
              "answers": [
                {
                  "text": "11월 7일",
                  "answer_start": 144
                }
              ],
              "guid": "52668e1e64614dfdb32656467e923202"
            }
          ]
        }
      ],
      "news_category": "게임",
      "source": "acrofan"
    },
    {
      "title": "삼성, 日샤프에 자본수혈",
      "paragraphs": [
        {
          "context": "삼성전자가 경영난을 겪고 있는 일본 샤프에 출자해 지분 3%를 확보한다. 일본에서 기술을 수입해 커온 삼성전자가 일본 경쟁 업체에 자본을 투자하는 첫 사례다. ▶관련기사 A3면삼성전자는 60인치 이상 대형 LCD TV 패널의 안정적 확보를 위한 것이라고 설명했다. 애플 견제 등 다목적 카드로 쓰일 수 있다는 관측도 나온다. 삼성전자는 6일 샤프에 104억엔(약 1200억원)을 출자하겠다고 발표했다. 니혼게이자이신문 등 일본 언론들도 이 사실을 일제히 보도했다. 샤프는 이달 중 제3자 배정 유상증자 형식으로 지분 3%를 삼성전자 일본 법인에 넘기기로 했다. 삼성이 지분 3% 인수를 마무리하면 니혼생명 미즈호코퍼레이트은행 등 금융사들에 이어 다섯 번째 대주주가 된다. 금융회사를 제외하면 삼성은 샤프의 최상위 주주로 올라선다.삼성전자는 출자 목적을 ‘TV 패널의 안정적 공급처 확보’라면서 경영에는 관여하지 않을 것이라고 밝혔다. 삼성전자는 2011년부터 10세대 액정표시장치(LCD) 공장을 보유한 샤프로부터 60인치 TV 패널을 공급받고 있다. 전체 TV 패널 수요량의 10% 이상을 샤프에서 받고 있는 것으로 알려졌다. 샤프는 출자를 받아 재무 상황을 개선하는 한편 삼성전자에 대한 TV 패널 공급량도 확대해 공장 가동률을 높일 계획인 것으로 알려졌다. 샤프는 올 3월 말 마감하는 2012 회계연도에 4500억엔의 적자가 예상되는 등 2년 연속 손실을 낼 것이라는 게 시장의 추정이다. 오는 9월까지 2000억엔 규모의 회사채 상환도 예정돼 있어 증자가 시급한 상황이다. 샤프는 작년 3월 대만 전자업체 훙하이로부터 669억엔을 출자받기로 합의했으나 진전을 이루지 못했다. 도쿄=안재석 특파원/김현석 기자",
          "qas": [
            {
              "question": "삼성전자가 LCD 공장을 보유한 회사에 출자하는 금액은?",
              "answers": [
                {
                  "text": "104억엔",
                  "answer_start": 195
                },
                {
                  "text": "104억엔(약 1200억원)",
                  "answer_start": 195
                },
                {
                  "text": "약 1200억원",
                  "answer_start": 201
                }
              ],
              "guid": "bf9d9b2212cc437ead8cfc892a76ac64"
            },
            {
              "question": "삼성전자가 이번 투자로 샤프에서 공급받으려는 부품은 어떤 가전제품의 것인가?",
              "answers": [
                {
                  "text": "60인치 이상 대형 LCD TV",
                  "answer_start": 104
                },
                {
                  "text": "대형 LCD TV",
                  "answer_start": 112
                },
                {
                  "text": "LCD TV",
                  "answer_start": 115
                },
                {
                  "text": "TV",
                  "answer_start": 119
                }
              ],
              "guid": "00a98de27a63414d9af4713431daa51c"
            }
          ]
        }
      ],
      "news_category": "종합",
      "source": "hankyung"
    },
    {
      "title": "강태진·김명환·성낙인·오세정·조동성 … ‘서울대 총장 후보’ 압축 ...“국가정책 이끄는 대학으로” 5人5色 공약대결",
      "paragraphs": [
        {
          "context": "서울대 차기 총장직에 나선 총장예비후보자가 5명으로 압축됐다. 서울대 총장추천위원회는 3일 오후 제5차 회의를 열고 후보대상자 12명 가운데 강태진 재료공학부 교수, 김명환 수리과학부 교수, 성낙인 법학전문대학원 교수, 오세정 물리천문학부 교수, 조동성 경영대학 명예교수 등 5명(가나다순)을 총장예비후보자로 선정했다고 발표했다.총장 후보자가 5명으로 좁혀지면서 법인화 이후 처음 간선제로 치러지는 서울대 총장 선거 레이스가 본격화됐다.○“서울대 위상 되찾자”서울대가 지향할 미래비전과 관련해 예비후보들은 서울대가 앞으로 국가의 공공정책 방향을 제시할 수 있어야 한다고 강조했다. 오 교수는 “서울대는 한국사회 지식생태계 전반을 선도하는 정책지식 산출의 중심이 돼야 한다”며 ‘정책지식연구원’ 설립안을 내놨다. 새로 만들어질 시흥캠퍼스에 ‘국가미래전략연구원’을 설립하자는 공약도 나왔다. 강 교수는 “국가의 미래를 위한 정책대안을 제시하고 사회문제 해법을 연구하는 기관을 세우자”고 제안했다. 조 명예교수는 “‘미래연구기획단’을 설치하고 통일학 등 국가 아젠다 연구를 서울대가 주도해야 한다”고 말했다.○“학부교육 강화”후보들은 서울대가 그간 ‘연구중심대학’을 지향하며 소홀히 했던 학부교육을 강화해야 한다고 목소리를 높였다. 김 교수는 “교수들이 승진이나 연구비를 따기 위해 단기성과에 치중하는 사이 학생들은 취업준비와 학점경쟁에 내몰렸다”고 지적했다. 강 교수는 “기초교육을 대폭 강화하겠다”며 획일적인 제2전공 의무제를 폐지하고 리더십 교육을 하겠다는 공약을 내놓았다. 거주형 학부대학(Residential College)을 도입하자는 주장도 나왔다. 오 교수는 “학부교육에서 세계시민적 인재를 양성하겠다”며 관악캠퍼스 내 거주형 대학을 도입하겠다고 약속했다.교양전담교수제 도입을 주장한 후보자도 있다. 조 명예교수는 “65세 정년퇴임 교수들이 5년간 학부 교양과목을 가르치도록 초빙하겠다”는 공약을 내걸었다. 성 교수는 “발전기금 모금과 자산 관리를 담당할 재무부총장(CFO)직을 신설하겠다”고 약속했다.○25일 정책평가 실시총추위는 이날 총장후보자 12명을 대상으로 1인당 20분의 소견발표 및 질의응답과정을 거쳐 이들을 예비후보자로 선정했다. 이후 공개소견발표와 정책평가가 진행된다. 공개소견발표는 연건캠퍼스와 관악캠퍼스에서 각각 오는 16일과 18일 오후 2~6시에 열린다. 교수 222명과 직원 22명 등 총 244명으로 구성된 정책평가단은 오는 25일 오후 관악캠퍼스에서 예비후보자들의 교육·연구 등 정책 및 실현 가능성, 비전과 리더십, 국제적 안목 등을 평가하게 된다. 9명의 위원으로 구성된 검증소위원회는 △상벌 관련 내용 △경력 및 자료 진위 관련 내용 △표절 등 연구윤리 위반 관련 내용 등을 검증한다.",
          "qas": [
            {
              "question": "서울대학교 다음 총장으로 출마한 후보 수는?",
              "answers": [
                {
                  "text": "12명",
                  "answer_start": 1040
                },
                {
                  "text": "12",
                  "answer_start": 1040
                }
              ],
              "guid": "78a5bd385d2f4059901fc33239bf0c62"
            }
          ]
        }
      ],
      "news_category": "사회",
      "source": "hankyung"
    },
    {
      "title": "“SDR 편입 후 위안화 약세 가속”",
      "paragraphs": [
        {
          "context": "중국 위안화가 국제통화기금(IMF) 특별인출권(SDR) 구성통화로 편입됨에 따라 앞으로 위안화 가치가 어떤 방향으로 움직일지에 관심이 모아지고 있다. 위안화의 SDR 편입은 중장기적으로는 위안화 강세 요인이다. 위안화 자산에 대한 수요가 증가하면 위안화 가치는 강세를 보일 수밖에 없기 때문이다. 하지만 최근 한 달간 위안화 가치는 줄곧 약세를 보였다. 미국 달러화 대비 위안화 가치는 지난 10월 말 달러당 6.3181위안에서 지속적으로 떨어져 1일에는 6.3981위안(오후 3시 기준)까지 낮아졌다. 미국 중앙은행(Fed)이 이달 중에 기준금리를 인상할 것이란 기대로 미국 달러화가 강세를 보였기 때문이다.중국 중앙은행인 인민은행은 최근 몇 달간 위안화 가치 하락을 막기 위해 외환시장에 개입해왔다. 위안화가 약세 기조를 보이면 IMF의 SDR 구성통화로 편입되는 데 불리하게 작용할 것이란 우려에서였다. 하지만 위안화의 SDR 편입으로 인민은행이 위안화 가치를 인위적으로 높게 유지해야 할 필요성이 없어졌다. 월스트리트저널(WSJ)은 “중국 정부가 앞으로 수출 경쟁력 제고를 위해 위안화 가치의 완만한 하락을 유도할 수 있다”고 전망했다.인민은행은 그러나 이 같은 시장의 관측을 강하게 부정했다. 이강(易鋼) 중국 인민은행 부행장 겸 국가외환관리국장은 1일 위안화의 SDR 편입 관련 기자회견에서 “위안화 가치 하락에 대한 우려는 불필요하다”며 “중국은 기본적으로 합리적이고 균형 잡힌 수준에서 위안화 가치를 유지할 능력을 갖추고 있다”고 주장했다. 그는 “IMF가 SDR 편입을 위한 평가를 할 때도 위안화 가치를 논한 게 아니다”며 “현재 중국이 시행하고 있는 관리 변동환율제의 기본은 SDR 편입 후에도 변하지 않을 것”이라고 강조했다. 베이징=김동윤 특파원",
          "qas": [
            {
              "question": "위안화 가치가 떨어질것이라고 예측한 곳은?",
              "answers": [
                {
                  "text": "월스트리트저널",
                  "answer_start": 512
                }
              ],
              "guid": "7d11c267e2f54ae3ab6aac549d673a9e"
            },
            {
              "question": "WSJ의 전망을 부정한 은행은?",
              "answers": [
                {
                  "text": "중국 중앙은행",
                  "answer_start": 341
                },
                {
                  "text": "인민은행",
                  "answer_start": 584
                }
              ],
              "guid": "fc877f453b1c4103bb7a3adff90a7ac8"
            }
          ]
        }
      ],
      "news_category": "종합",
      "source": "hankyung"
    },
    {
      "title": "불꽃 소방대",
      "paragraphs": [
        {
          "context": "태양력 198년 도쿄. 이 세상은 어느 날을 경계로 갑자기 시작된 인체 발화현상에 의한 위협에 괴로워하고 있었다. 갑자기 자신의 몸에서 발화한 자는 순식간에 자아를 잃고 몸이 고통스럽게 불태워진다. 이들은 '화염인간'이라는 호칭으로 불리며 공포의 대상이 되고있다.\n\n이 화염인간들은 '제1세대'라 불리며 이 후 화염을 조종할수 있는 '제2세대'와 자신의 몸에서 발화해도 그 불꽃을 스스로 조종할 수 있는 '제3세대'가 나타난다.\n\n그리고 화염인간에 의한 위험에 대응해 일반 시민을 불꽃 공포로부터 지키고 인체 발화 현상의 원인과 해결책을 규명하기 위해 조직된 것이 특수 소방대다.\n\n어릴 적부터 영웅을 동경하던 주인공 '신라 쿠사카베'는 12년 전 갑작스런 화재로 어머니와 갓 태어난 남동생을 잃고 만다. 발에서 불꽃을 낼 수 있는 제3세대의 능력자 신라는 자신의 몸에서 발화한 불꽃이 화재원인이라고 해서 주위로부터 박해를 받는데, 신라 자신은 모친과 남동생 이외의 제3자가 실내에 있던 것을 목격하고, 그 인물이 범인이 아닌가 하고 생각한다.훈련교를 졸업하고 제8 특수 소방대에 배속된 신라는 어머니와 동생 같은 피해자를 늘리지 않기 위해, 또 어머니와 동생을 죽인 범인을 붙잡고 스스로 뒤집어쓴 누명을 벗기 위해 동료들과 함께 오늘도 훈련과 소방활동에 분투한다. 그 중 제8 특수 소방대가 결성된 목적을 알고, 화염인간의 비밀, 이를 추적하기 위해 적대자와 싸우거나, 다른 소방대를 조사하게 된다.",
          "qas": [
            {
              "question": "자신의 몸에서 발화가 가능한 사람들을 무엇이라고 부르나요?",
              "answers": [
                {
                  "text": "'화염인간'",
                  "answer_start": 115
                },
                {
                  "text": "화염인간",
                  "answer_start": 116
                }
              ],
              "guid": "006af6aea6da475f87276f1780b46dc6"
            },
            {
              "question": "신라가 불을 일으킬 수 있는 신체 부위는?",
              "answers": [
                {
                  "text": "발",
                  "answer_start": 395
                }
              ],
              "guid": "7e634920eb5b4b7bb369ef3e290d74bd"
            }
          ]
        }
      ],
      "news_category": "",
      "source": "wikipedia"
    }]}


test_sample = {
  "version": "Ko-MRC",
  "data": [
    {
      "title": "BMW 코리아, 창립 25주년 기념 ‘BMW 코리아 25주년 에디션’ 한정 출시",
      "paragraphs": [
        {
          "context": "BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.",
          "qas": [
            {
              "question": "말라카이트에서 나온 색깔을 사용한 에디션은?",
              "answers": "",
              "guid": "d14cb73158624cf094c546d856fd3c80"
            }
          ]
        }
      ],
      "news_category": "자동차",
      "source": "acrofan"
    }]}

NameError: ignored

## Data Load

데이터를 로드하고, parsing 해서 필요한 데이터만 뽑아주는 모듈 
KoMRC

In [12]:
from typing import List, Tuple, Dict, Any
import json
import random

class KoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, data):
        # with open(file_path, 'r', encoding='utf-8') as fd:
        #     data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        context = paragraph['context']
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        answers = qa['answers']

        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)

In [15]:
dataset = KoMRC.load(train_sample)
print("Number of Samples:", len(dataset))
pprint(dataset[0])

train_dataset, dev_dataset = KoMRC.split(dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Samples: 3
{'answers': [{'answer_start': 478, 'text': '한 달가량'},
             {'answer_start': 478, 'text': '한 달'}],
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 '
            '전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 '
            '호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 '
            '하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 '
            '내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 '
            '줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 '
            '장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 '
            '시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 '
            '전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, '
            '강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 '
            '내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 '
         

IndexError: ignored